In [1]:
import wntr
import networkx as nx

In [2]:
import pickle

In [34]:
Wdn = wntr.network.WaterNetworkModel('EXN.inp') 

In [35]:
Wdn.options.time.duration = 1 * 3600
Sim = wntr.sim.EpanetSimulator(Wdn)
Results = Sim.run_sim()

In [36]:
Flow = Results.link['flowrate']

In [37]:
def path_strength(graph, path):
        return sum([graph[path[i]][path[i+1]]['weight'] for i in range(len(path)-1)])

In [30]:
# Create hydraulic-informed network 
def creathydnet(wdn, flow, t): 
    # Build two directed graphs, dgraph: normal weight, rdgraph: reversed weight 
    dgraph = nx.DiGraph()
    rdgraph = nx.DiGraph()
    # Use link (pipe, valve) diameter as the main selection criteria
    for name, link in wdn.links():
        # Determine the direction of edge through the flow rate, positive: as stored, negative: change direction
        linkflow = flow.loc[3600*t, name]
        if linkflow >= 0:
            start_node = link.start_node_name
            end_node = link.end_node_name
        else:
            start_node = link.end_node_name
            end_node = link.start_node_name
        if link.link_type == 'Pipe':
            pidia = link.diameter
            linkweight = (abs(linkflow)*pidia**2)  
        # Pumps and valves have larger weight
        elif link.link_type == 'Pump' or link.link_type == 'Valve':
            linkweight = abs(linkflow)
        # Build corresponding original network
        if linkweight < 1e-6:
            reverseweight = 1e6
        else:
            reverseweight = 1/linkweight  
        dgraph.add_node(start_node, pos = wdn.get_node(start_node).coordinates)
        dgraph.add_node(end_node, pos = wdn.get_node(end_node).coordinates)
        dgraph.add_edge(start_node, end_node, linkid = name, weight = linkweight)
        rdgraph.add_edge(start_node, end_node,  weight = reverseweight)
    return dgraph, rdgraph 

In [31]:
# Calculate the local reaching centrality of each node
def calculateLRC(dgraph, rdgraph):
    lcrect = {}
    nodelist = list(dgraph.nodes)
    # For each node i, get its reachable nodes and the path length
    for nodei in nodelist:
        # Get its all downstream nodes
        dni = list(nx.algorithms.dag.descendants(dgraph, nodei))
        # For each (i,j) connection, calculate the weighted reachability
        cri = 0
        for nodej in dni:
            # Find all the paths (i,j) and get the one with larhest stength
            # Path is the the one with shortest for rdgraph 
            path = nx.algorithms.shortest_paths.generic.shortest_path(rdgraph, source=nodei, target=nodej, weight="weight")
            # Calculate the path length for dgraph
            weight_path_strength = path_strength(dgraph, path)
            path_length = len(path) - 1
            # Calculate the weighted contribution to local reaching centrality 
            cri += weight_path_strength/path_length
        # Store its local reaching centrality   
        lcrect[nodei] = cri/(len(nodelist)-1)
    return lcrect

In [38]:
T = 1
Dgraph, Rdgraph = creathydnet(Wdn, Flow, T)

In [39]:
LRC = calculateLRC(Dgraph, Rdgraph)

In [40]:
# Save the LRC to pickle 
FLRC = open(r"%s.pickle" % ('EXN_LRC'), 'wb')
pickle.dump(LRC, FLRC)
FLRC.close()